### Search Engine With Tools And Agents

### Explanation of the Code
#### Importing Necessary Modules
The code imports necessary modules from `langchain_community`:
* `ArxivQueryRun` and `WikipediaQueryRun` from `langchain_community.tools`: these classes are used to run queries on Arxiv and Wikipedia, respectively.
* `WikipediaAPIWrapper` and `ArxivAPIWrapper` from `langchain_community.utilities`: these classes provide a wrapper around the Wikipedia and Arxiv APIs, respectively, to interact with these services.

#### Purpose of the Modules
* `ArxivQueryRun`: allows running queries on the Arxiv database, which is a repository of electronic preprints in physics, mathematics, computer science, and related disciplines.
* `WikipediaQueryRun`: allows running queries on Wikipedia, which is a free online encyclopedia.
* `WikipediaAPIWrapper`: provides a convenient interface to interact with the Wikipedia API, allowing for tasks such as searching for articles, retrieving article content, and more.
* `ArxivAPIWrapper`: provides a convenient interface to interact with the Arxiv API, allowing for tasks such as searching for preprints, retrieving preprint metadata, and more.

### Use Cases
These modules can be used in a variety of applications, such as:
* Building a conversational AI that can provide information on scientific topics by querying Arxiv and Wikipedia.
* Developing a research assistant that can help with literature reviews by searching for relevant preprints on Arxiv and articles on Wikipedia.
* Creating a knowledge graph that integrates information from Arxiv and Wikipedia to provide a comprehensive view of a particular topic or field.

In [17]:
## Arxiv--Research
## Tools creation
from langchain_community.tools import ArxivQueryRun,WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper,ArxivAPIWrapper

In [18]:
## Used the inbuilt tool of wikipedia
api_wrapper_wiki=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=250)
wiki=WikipediaQueryRun(api_wrapper=api_wrapper_wiki)
wiki.name

'wikipedia'

In [19]:
api_wrapper_arxiv=ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=250)
arxiv=ArxivQueryRun(api_wrapper=api_wrapper_arxiv)
print(arxiv.name)

arxiv


In [20]:
tools=[wiki,arxiv]

In [21]:
## Custom tools[RAG Tool]
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [27]:
import os
from dotenv import load_dotenv
load_dotenv()
os.environ['OPENAI_API_KEY']=os.getenv('OPENAI_API_KEY')

In [28]:
loader=WebBaseLoader("https://docs.smith.langchain.com/")
docs=loader.load()
documents=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)
vectordb=FAISS.from_documents(documents,OpenAIEmbeddings())
retriever=vectordb.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000002117A748340>, search_kwargs={})

In [13]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool=create_retriever_tool(retriever,"langsmith-search","Search any information about Langsmith ")

retriever_tool.name

'langsmith-search'

In [14]:
tools=[wiki,arxiv,retriever_tool]

In [15]:
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'c:\\Users\\DELL\\miniconda3\\envs\\agenticai_env\\lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=250)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=250)),
 Tool(name='langsmith-search', description='Search any information about Langsmith ', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x0000021155639FC0>, retriever=VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000021

In [16]:
## Run all this tools with Agents and LLM Models

## Tools, LLM-->AgentExecutor
from langchain_groq import ChatGroq
from dotenv import load_dotenv
import openai
load_dotenv()
import os

groq_api_key=os.getenv("GROQ_API_KEY")
openai.api_key=os.getenv("OPENAI_API_KEY")

llm=ChatGroq(groq_api_key=groq_api_key,model_name="Llama3-8b-8192")

### Explanation of the Code
#### Importing the Hub Module
The code imports the `hub` module from `langchain`, which provides a way to access and interact with pre-built language models and agents.

#### Pulling a Pre-Built Agent
The `hub.pull` function is used to retrieve a pre-built agent from the LangChain hub. In this case, the agent is identified by the username `hwchase17` and the agent name `openai-functions-agent`.

#### Accessing the Agent's Messages
The `prompt.messages` attribute is used to access the messages associated with the pulled agent. This can include the agent's capabilities, instructions, or other relevant information.

### Purpose of the Code
The purpose of this code is to retrieve a pre-built agent from the LangChain hub and access its associated messages. This can be useful for a variety of applications, such as:
* Using pre-built agents to perform specific tasks or functions
* Customizing or fine-tuning pre-built agents for particular use cases
* Integrating pre-built agents into larger language models or applications

### Benefits of Using Pre-Built Agents
Using pre-built agents from the LangChain hub can provide several benefits, including:
* **Time savings**: pre-built agents can save time and effort by providing a pre-trained and pre-configured solution
* **Expertise**: pre-built agents can be created by experts in a particular domain or task, providing access to specialized knowledge and capabilities
* **Consistency**: pre-built agents can provide a consistent and reliable way to perform specific tasks or functions, reducing errors and variability.

In [11]:
## Prompt Template
from langchain import hub
prompt=hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

### Explanation of the Code
#### Importing the Create Agent Function
The code imports the `create_openai_tools_agent` function from `langchain.agents`, which is used to create an agent that utilizes OpenAI tools.

#### Creating an Agent
The `create_openai_tools_agent` function is used to create an agent that combines the capabilities of a large language model (`llm`) with a set of tools (`tools`) and a prompt (`prompt`).

#### Agent Components
The agent consists of the following components:
* `llm`: a large language model that provides the core language understanding and generation capabilities
* `tools`: a set of tools that can be used by the agent to perform specific tasks or functions
* `prompt`: a prompt that defines the context and goals for the agent's actions

### Purpose of the Code
The purpose of this code is to create an agent that can leverage the capabilities of a large language model and a set of tools to perform complex tasks and functions.

### Benefits of the Agent
The created agent can provide several benefits, including:
* **Improved task completion**: the agent can use the tools to perform specific tasks and functions, improving the overall task completion rate
* **Enhanced language understanding**: the large language model provides the agent with advanced language understanding capabilities, allowing it to better comprehend and respond to user input
* **Flexibility and adaptability**: the agent can be easily adapted to different tasks and domains by changing the tools and prompt

### Potential Use Cases
The created agent can be used in a variety of applications, such as:
* **Virtual assistants**: the agent can be used to create virtual assistants that can perform tasks and answer questions on behalf of users
* **Chatbots**: the agent can be used to create chatbots that can engage in conversations with users and provide helpful responses
* **Language-based interfaces**: the agent can be used to create language-based interfaces for applications and systems, providing users with a natural and intuitive way to interact with technology.

In [12]:
## Agents
from langchain.agents import create_openai_tools_agent
agent=create_openai_tools_agent(llm,tools,prompt)
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages

### Explanation of the Code
#### Importing the AgentExecutor Class
The code imports the `AgentExecutor` class from `langchain.agents`, which is used to execute an agent's actions and provide a way to interact with the agent.

#### Creating an AgentExecutor Instance
An instance of the `AgentExecutor` class is created, passing in the following parameters:
* `agent`: the agent to be executed, which was created earlier
* `tools`: the set of tools that the agent can use to perform tasks and functions
* `verbose`: a flag that controls the level of output and feedback provided by the agent executor (set to `True` in this case)

#### Purpose of the AgentExecutor
The `AgentExecutor` instance provides a way to execute the agent's actions and interact with the agent, allowing for the following benefits:
* **Execution of agent actions**: the agent executor can execute the agent's actions and provide feedback on the outcome
* **Tool usage**: the agent executor can use the provided tools to perform specific tasks and functions
* **Verbose output**: the agent executor can provide detailed output and feedback on the agent's actions and decisions

### Benefits of Using an AgentExecutor
Using an `AgentExecutor` instance can provide several benefits, including:
* **Simplified agent interaction**: the agent executor provides a simple and intuitive way to interact with the agent and execute its actions
* **Improved feedback and output**: the agent executor can provide detailed feedback and output on the agent's actions and decisions, making it easier to understand and debug the agent's behavior
* **Flexible tool usage**: the agent executor can use a variety of tools to perform tasks and functions, making it a flexible and adaptable solution for different applications and domains

### Potential Use Cases
The `AgentExecutor` instance can be used in a variety of applications, such as:
* **Agent-based systems**: the agent executor can be used to create agent-based systems that can perform complex tasks and functions
* **Virtual assistants**: the agent executor can be used to create virtual assistants that can interact with users and perform tasks on their behalf
* **Autonomous systems**: the agent executor can be used to create autonomous systems that can operate independently and make decisions based on their environment and goals.

In [13]:
## Agent Executer
from langchain.agents import AgentExecutor
agent_executor=AgentExecutor(agent=agent,tools=tools,verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag

In [14]:
agent_executor.invoke({"input":"Tell me about Langsmith"})



> Entering new AgentExecutor chain...

Invoking: `langsmith-search` with `{'query': 'Langsmith'}`


Get started with LangSmith | 🦜️🛠️ LangSmith

LangSmith + LangChain OSSLangSmith integrates seamlessly with LangChain's open source frameworks langchain and langgraph, with no extra instrumentation needed.If you're already using either of these, see the how-to guide for setting up LangSmith with LangChain or setting up LangSmith with LangGraph.
Observability​
Observability is important for any software application, but especially so for LLM applications. LLMs are non-deterministic by nature, meaning they can produce unexpected results. This makes them trickier than normal to debug.
This is where LangSmith can help! LangSmith has LLM-native observability, allowing you to get meaningful insights from your application. LangSmith’s observability features have you covered throughout all stages of application development - from prototyping, to beta testing, to production.

Skip to main conten

{'input': 'Tell me about Langsmith',
 'output': 'The Langsmith platform seems to be a suite of tools and services for building, monitoring, and evaluating Large Language Model (LLM) applications. It focuses on providing observability, evaluation, and prompt engineering capabilities to help developers create high-quality and production-grade LLM applications.\n\nSome of the key features of Langsmith include:\n\n1. Observability: Langsmith provides LLM-native observability, allowing developers to track and analyze the behavior of their applications. This includes features like tracing, metrics, dashboards, and alerts.\n2. Evaluation: Langsmith offers evaluation tools to help developers assess the performance of their applications. This includes features like automatic scoring, human feedback, and comparison of results over time.\n3. Prompt Engineering: Langsmith provides tools for prompt engineering, which involves writing prompts to instruct the LLM on what to do. This includes features

In [15]:
agent_executor.invoke({"input":"What is machine learning"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Machine learning'}`


Page: Machine learning
Summary: Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data, and thus perform tasks witThank you for providing the output from the tool call. Based on the provided information, I can provide a more detailed answer to your question.

Machine learning is a subset of artificial intelligence (AI) that involves the use of algorithms and statistical models to enable machines to learn from data, make decisions, and improve their performance over time. It's a type of self-learning, where the machine learns from the data it's given, without being explicitly programmed.

Machine learning algorithms can be categorized into three types:

1. Supervised learning: This type of learning involves training the algorithm on labeled data, where 

{'input': 'What is machine learning',
 'output': "Thank you for providing the output from the tool call. Based on the provided information, I can provide a more detailed answer to your question.\n\nMachine learning is a subset of artificial intelligence (AI) that involves the use of algorithms and statistical models to enable machines to learn from data, make decisions, and improve their performance over time. It's a type of self-learning, where the machine learns from the data it's given, without being explicitly programmed.\n\nMachine learning algorithms can be categorized into three types:\n\n1. Supervised learning: This type of learning involves training the algorithm on labeled data, where the output is already known. The algorithm learns to map inputs to outputs based on the labeled data.\n2. Unsupervised learning: In this type of learning, the algorithm is trained on unlabeled data, and it must find patterns or structure on its own.\n3. Reinforcement learning: This type of learn

In [16]:
agent_executor.invoke({"input":"What's the paper 1706.03762 about?"})



> Entering new AgentExecutor chain...

Invoking: `arxiv` with `{'query': '1706.03762'}`


Published: 2023-08-02
Title: Attention Is All You Need
Authors: Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz Kaiser, Illia Polosukhin
Summary: The dominant sequence transduction models are based on c
Invoking: `wikipedia` with `{'query': 'Attention Is All You Need'}`


Page: Attention Is All You Need
Summary: "Attention Is All You Need" is a 2017 landmark research paper in machine learning authored by eight scientists working at Google. The paper introduced a new deep learning architecture known as the transformer,The paper "Attention Is All You Need" is a 2017 landmark research paper in machine learning authored by eight scientists working at Google. The paper introduced a new deep learning architecture known as the transformer, which is primarily used for natural language processing tasks such as machine translation, text summarization, and la

{'input': "What's the paper 1706.03762 about?",
 'output': 'The paper "Attention Is All You Need" is a 2017 landmark research paper in machine learning authored by eight scientists working at Google. The paper introduced a new deep learning architecture known as the transformer, which is primarily used for natural language processing tasks such as machine translation, text summarization, and language modeling. The transformer has since been widely adopted in many areas of artificial intelligence and has achieved state-of-the-art results in many tasks.'}